In [28]:
import os
import subprocess
from datetime import datetime
import numpy as np
import pickle

In [29]:
def get_last_commit_date(file_path, clone_path):
    try:
        # Use '--' to separate the file path
        output = subprocess.check_output(['git', 'log', '-1', '--format=%cd', '--', file_path], cwd=clone_path)
        return output.strip().decode('utf-8')
    except subprocess.CalledProcessError as e:
        print(f"Error processing file {file_path}: {e}")
        return None


In [30]:
clone_path = '/home/skunk/efficbench/alpaca_eval'
models_path = clone_path+'/results'
models_names = [name for name in os.listdir(models_path) if os.path.isdir(os.path.join(models_path, name))]

In [31]:
!pwd

/home/skunk/efficbench


In [32]:
dates={}
for model in models_names:
    file_path = models_path+f'/{model}/model_outputs.json'
    dates[model]=(datetime.strptime(get_last_commit_date(file_path, clone_path), '%a %b %d %H:%M:%S %Y %z'))

In [33]:
with open('data/alpaca_dates.pickle', 'wb') as handle:
    pickle.dump(dates, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [34]:
with open('data/alpaca_v2.pickle', 'rb') as handle:
    data = pickle.load(handle)
    
#post-process (might be included in the pre-process code later on)
#ordering by including date and deleting benchmark model (gpt4 turbo, which is the best performing model)
with open('data/alpaca_dates.pickle', 'rb') as handle:
    alpaca_dates = pickle.load(handle)
dates = [alpaca_dates[model] for model in data['models']]
order = np.argsort(np.array(dates))[::-1]

data['data']['alpaca_v2']['correctness'] = data['data']['alpaca_v2']['correctness'][:,order]
benchmark_ind = np.argmax(data['data']['alpaca_v2']['correctness'].mean(axis=0))
data['data']['alpaca_v2']['correctness'] = np.delete(data['data']['alpaca_v2']['correctness'], benchmark_ind, axis=1)

data['models'] = np.array(data['models'])[order].tolist()
data['models'].pop(benchmark_ind)

'gpt4_turbo'

In [35]:
with open('data/alpaca_v2_processed.pickle', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [38]:
np.array(dates)[order][:int(98/4)]

array([datetime.datetime(2024, 1, 17, 23, 28, 33, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=57600))),
       datetime.datetime(2024, 1, 16, 11, 36, 30, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=57600))),
       datetime.datetime(2024, 1, 16, 1, 34, 36, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=57600))),
       datetime.datetime(2024, 1, 11, 12, 1, 34, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=57600))),
       datetime.datetime(2024, 1, 10, 15, 13, 3, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=57600))),
       datetime.datetime(2024, 1, 10, 13, 9, 5, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=57600))),
       datetime.datetime(2024, 1, 10, 13, 9, 5, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=57600))),
       datetime.datetime(2024, 1, 10, 13, 9, 5, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=57600))),
       datetime.datetime(2024, 1, 10, 13,